In [3]:
import os 

In [4]:
os.chdir("../")

In [17]:
%pwd

'/Users/sanahaidar/Desktop/NLP/text-summarizer'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int 

In [19]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])
    
    # Model Trainer Configuration
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer
     

        create_directories([Path(config.root_dir)])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        

In [21]:
import os
from pathlib import Path
from textSummarizer.logging import logger
from transformers import AutoTokenizer,DataCollatorForSeq2Seq, TrainingArguments, Trainer,AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch


In [43]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        

    def train(self):
        device = torch.device("cpu")
        tokenizer= AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)



        dataset_samsum_pt= load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
    output_dir=self.config.root_dir,
    num_train_epochs=self.config.num_train_epochs,
    warmup_steps=self.config.warmup_steps,
    per_device_train_batch_size=self.config.per_device_train_batch_size,
    weight_decay=self.config.weight_decay,
    logging_steps=self.config.logging_steps,
    do_eval=True,
    save_steps=self.config.save_steps,
    gradient_accumulation_steps=self.config.gradient_accumulation_steps,
    no_cuda= True,  # Set to True to use CPU
)

        trainer = Trainer(model= model_pegasus,
                          args=trainer_args,
                          tokenizer=tokenizer,
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["train"],
                          eval_dataset= dataset_samsum_pt["validation"]
                          )

        trainer.train()


        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [44]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    trainer = ModelTrainer(config=model_trainer_config)
    trainer.train()

except Exception as e:
    raise e

[2025-07-29 21:08:13,664: INFO: common: YAML file config/config.yaml read successfully.]
[2025-07-29 21:08:13,668: INFO: common: YAML file params.yaml read successfully.]
[2025-07-29 21:08:13,669: INFO: common: Created directory: artifacts]
[2025-07-29 21:08:13,670: INFO: common: Created directory: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/sanahaidar/Desktop/NLP/text-summarizer/newenv/lib/python3.12/site-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/ym/9nvn6tw55w3b0k7h583z055c0000gn/T/ipykernel_27034/3897822536.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model= model_pegasus,


Step,Training Loss
10,3.254000
20,2.844000
30,2.910400
40,3.101100
50,2.548800
60,2.680400
70,3.061600
80,2.576400
90,2.525000
100,2.733700


/Users/sanahaidar/Desktop/NLP/text-summarizer/newenv/lib/python3.12/site-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
